In [117]:
from utils.sentiment_analysis import SentimentAnalysis
from utils.translator import Translator
from utils.scraper import call_url_get_bs4
from bs4 import BeautifulSoup
from requests.structures import CaseInsensitiveDict
import requests
import time
from datetime import datetime, timedelta
import re
import pandas as pd
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from utils.db_access import DatabaseAccess


requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [118]:
db = DatabaseAccess()
datasets = db.get_rows("SELECT identifier, description FROM dataset")

Connection to 'stonksdb' made successfully


c:\Users\madsh\OneDrive\Code\Python\BiksStonks\data_collectors\python\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [119]:
datasets.iloc[1].description

'Columbus A/S'

In [120]:
ds = [[datasets.iloc[i].identifier, datasets.iloc[i].description] for i in range(len(datasets['description']))]

In [121]:
ds

[[15518, 'Coloplast B A/S'],
 [15520, 'Columbus A/S'],
 [745915, 'Danske Andelskassers Bank A/S'],
 [15521, 'Danske Bank A/S'],
 [15645, 'Dantax A/S'],
 [15530, 'Djurslands Bank A/S'],
 [24490, 'FirstFarms A/S'],
 [15489, 'Alm. Brand A/S'],
 [19782, 'ALK-Abelló B A/S'],
 [15554, 'Jyske Bank A/S'],
 [405195, 'Matas A/S'],
 [9800892, 'Netcompany Group A/S'],
 [15567, 'NKT A/S'],
 [10900494, 'Nordea Bank Abp'],
 [15629, 'Novo Nordisk B A/S'],
 [167, 'Novozymes A/S'],
 [15129047, 'NTG Nordic Transport Group'],
 [15254, 'Ambu A/S'],
 [23681181, 'Aquaporin A/S'],
 [2086, 'Bavarian Nordic A/S'],
 [34861, 'BioPorto A/S'],
 [23632, 'Atlantic Petroleum P/F'],
 [15623, 'Brd. Klee B A/S'],
 [15515, 'Carlsberg A A/S'],
 [15516, 'Carlsberg B A/S'],
 [7070000, 'Copenhagen Capital A/S'],
 [15449923, 'EAC Invest A/S'],
 [38088, 'Bang & Olufsen Holding A/S'],
 [15538, 'Flugger Group A/S'],
 [162, 'H. Lundbeck A/S'],
 [8350289, 'Orphazyme A/S'],
 [48755, 'Pandora A/S'],
 [15637, 'Rockwool International A

In [122]:
custome_names = {
    "Netcompany Group" : ['Netcompany'],
    'Nordea Bank Abp':['Nordea Bank'],
    'Atlantic Petroleum P/F':['Atlantic Petroleum'],
    'Bang & Olufsen Holding':['Bang & Olufsen', 'Bang og Olufsen', 'B&O'],
    'Flugger Group':['Flügger'],
    'Rockwool International':['Rockwool'],
    'SAS AB' : ['SAS'],
    'Spar Nord Bank':['Spar Nord'],
    'TORM Plc': ['TORM'],
    'Vestas Wind Systems': ['Vestas'],
    'BankNordik P/F':['BankNordik'],
    'Boozt AB':['Boozt'],
    'Aalborg Boldspilklub':['aab'],
    'Harboes Bryggeri':['Harboe'],
    'Københavns Lufthavne':['Københavns Lufthavn'],
    'Lån og Spar Bank':['Lån og Spar'],
    'Onxeo SA (Copenhagen)':['Onxeo SA'],
    'Silkeborg IF Invest':['Silkeborg IF', 'SIF'],
    'A.P. Møller - Mærsk':['Mærsk', 'A.P. Møller', 'Maersk'],
    'Trifork Holding':['Trifork', 'Trifork Holding'],
    'Brøndbyernes IF':['Brøndby IF', 'BIF'],
    'PARKEN Sport & Entertainment':['parken']
}

In [123]:
for d in ds:
    if ' A/S' in d[1]:
        d[1] = d[1].replace(' A/S', '')
    if d[1][-2:] == ' B':
        d[1] = d[1][:-2]
    if d[1][-2:] == ' A':
        d[1] = d[1][:-2]
    if d[1] in custome_names:
        d.extend(custome_names[d[1]])
    
ds

[[15518, 'Coloplast'],
 [15520, 'Columbus'],
 [745915, 'Danske Andelskassers Bank'],
 [15521, 'Danske Bank'],
 [15645, 'Dantax'],
 [15530, 'Djurslands Bank'],
 [24490, 'FirstFarms'],
 [15489, 'Alm. Brand'],
 [19782, 'ALK-Abelló'],
 [15554, 'Jyske Bank'],
 [405195, 'Matas'],
 [9800892, 'Netcompany Group', 'Netcompany'],
 [15567, 'NKT'],
 [10900494, 'Nordea Bank Abp', 'Nordea Bank'],
 [15629, 'Novo Nordisk'],
 [167, 'Novozymes'],
 [15129047, 'NTG Nordic Transport Group'],
 [15254, 'Ambu'],
 [23681181, 'Aquaporin'],
 [2086, 'Bavarian Nordic'],
 [34861, 'BioPorto'],
 [23632, 'Atlantic Petroleum P/F', 'Atlantic Petroleum'],
 [15623, 'Brd. Klee'],
 [15515, 'Carlsberg'],
 [15516, 'Carlsberg'],
 [7070000, 'Copenhagen Capital'],
 [15449923, 'EAC Invest'],
 [38088, 'Bang & Olufsen Holding', 'Bang & Olufsen', 'Bang og Olufsen', 'B&O'],
 [15538, 'Flugger Group', 'Flügger'],
 [162, 'H. Lundbeck'],
 [8350289, 'Orphazyme'],
 [48755, 'Pandora'],
 [15637, 'Rockwool International', 'Rockwool'],
 [5864, 

In [124]:
cookies = {
    
}

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36", 
}

In [125]:
soup = call_url_get_bs4(f"https://api.borsen.dk/nyheder/side/5000", cookies=cookies, headers=headers)



In [126]:
s = soup.find_all('div', {'class':'col offset-md-2 body'})

In [127]:
len(s), len(soup.find_all('li'))

(100, 136)

In [128]:
months = {
    'jan':1,
    'feb':2,
    'mar':3,
    'apr':4,
    'maj':5,
    'jun':6,
    'jul':7,
    'aug':8,
    'sep':9,
    'okt':10,
    'nov':11,
    'dec':12
}

In [129]:
def parse_date(date):
    split_date = date.split(' ')
    day = int(split_date[0])
    month = int(months[split_date[1]])
    year = int(split_date[2])
    hour = int(split_date[3])
    minute = int(split_date[4])
    
    return datetime(year=year, month=month, day=day, hour=hour, minute=minute)

In [130]:
articels = pd.DataFrame(columns=['release_date', 'source_headline', 'target_headline', 'source_language', 'target_language', 'neg', 'pos', 'neu', 'compoud', 'url', 'companies'])
articels.head()

,release_date,source_headline,target_headline,source_language,target_language,neg,pos,neu,compoud,url,companies


In [131]:
def _add_row(articels, date, headline, url, companies):
    row = pd.DataFrame({'release_date' : [date], 
                        'source_headline': [headline],
                        'source_language' : ['da'],
                        'url' : [url],
                        'companies': [",".join([str(x) for x in companies])]})
    return pd.concat([articels, row], ignore_index = True, axis = 0)

In [132]:
def _get_date(mydivs):
    date = mydivs[0].get_text()
    date=date.replace(".", ' ').replace(':', ' ').replace('KL', ' ').replace('    ', ' ').replace('  ', ' ')
    return parse_date(date)

In [133]:
def _date_with_correct_format_exists(mydivs):
    return len(mydivs) > 0 and re.findall('[0-9]*\. [a-z]+ [0-9]* KL. [0-9]*:[0-9]*', mydivs[0].get_text())

In [134]:
def _get_companies_in_headlinen(company_names, headline):
    companines_in_headline = []
    for company in company_names:
        for c in company[1:]:
            if c.lower() in headline.lower():
                companines_in_headline.append(company[0])
                break
    return companines_in_headline

In [135]:
def parse_articels_in_page(page, company_names):
    articels = pd.DataFrame(columns=['release_date', 'source_headline', 'target_headline', 'source_language', 'target_language', 'neg', 'pos', 'neu', 'compoud', 'url', 'companies'])
    for link in page:
        while True:
            try:
                if not link.a == None:
                    url = link.a.get('href')
                    headline = link.a.get_text()
                    inner_soup = call_url_get_bs4(url, cookies=cookies, headers=headers)
                    
                    mydivs = inner_soup.find_all("span", {"class": "published"})

                    if _date_with_correct_format_exists(mydivs):
                        date = _get_date(mydivs)
                        companies = _get_companies_in_headlinen(company_names, headline)
                        articels = _add_row(articels, date, headline, url, companies)
                    else:
                        print(f"error - {headline} - {url} - {mydivs}")
                    break
            except Exception as e:
                print(f"Error {str(e)}")
                print("retrying in a few seconds")
                time.sleep(5)
    return articels


In [136]:
i = 1 #5515

company_names = ds

while True:
    try:
        soup = call_url_get_bs4(f"https://api.borsen.dk/nyheder/side/{i}", cookies=cookies, headers=headers)
        articels = soup.find_all('div', {'class':'col offset-md-2 body'})
        if len(articels) == 0:
            break
        else:
            articels_dataframe = parse_articels_in_page(articels, company_names)
            i += 1
            break
    except Exception as e:
        print(f"exception at {i} - {str(e)}")
        time.sleep(5)

print("done")

i


done


2

In [137]:
articels_dataframe[['source_headline', 'companies']].head(20)

,source_headline,companies
0,Svensk byggekæde brager ind i Danmark: Vil åbn...,
1,"Drømmehuset ved fjorden kom til salg, lige da ...",
2,Besøg modemilliardærens personlige kunstsamlin...,
3,Netto-direktør bliver ny topchef for energisel...,
4,Rådgivergigant lander rekordregnskab efter mas...,
5,Banknordiks topchef stopper – fortsætter indti...,27453
6,Rusland nægter krigsforbrydelser i Butja: Skyd...,
7,Debat | Business Danmark: Arbejdsgiverne må ik...,
8,Arving stopper som formand for Danfoss-ejerfon...,
9,Russiske aktier stiger igen: Vestlige aktører ...,


In [143]:
msg = "afganistanagf"

re.search(r'\b' + 'agf' + r'\b', msg)